In [79]:
#%pip install altair vega vega_datasets pandas

## Visualization Project: Movie Recommendation System

#### Dataset: The Movies Dataset
This system uses the movies dataset (https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset). It has metadata on over 45,000 movies. 26 million ratings from over 270,000 users.

#### Objective
The objective of this project is to build a movie recommendation system using the metadata of the movies dataset. The system will recommend movies with the appropriate visualizations based on the user's preferences.

In [80]:
import pandas as pd
import altair as alt
from ast import literal_eval
movies_metadata = pd.read_csv('./The_Movies_Dataset/movies_metadata.csv', low_memory=False)

In [81]:
movies_metadata_genres = movies_metadata[['id', 'title', 'genres']].copy()

# convert the genres column to a list
movies_metadata_genres['genres'] = (
    movies_metadata_genres['genres']
    .fillna('[]')
    .apply(literal_eval)
    .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
)

In [82]:
# Grouping by genres
genres = movies_metadata_genres.explode('genres')
genres = genres.groupby('genres').size().reset_index(name='count')
genres = genres.sort_values('count', ascending=False)

# calculate the percentage
genres['percentage'] = genres['count'] / genres['count'].sum() * 100

In [83]:
# plot the genres distribution
alt.Chart(genres).mark_arc().encode(
    theta=alt.Theta(field='percentage', type='quantitative', stack=True),
    color=alt.Color(field='genres', type='nominal'),
    tooltip=[alt.Tooltip('genres', title='Genre'), alt.Tooltip('percentage', format='.2f', title='Percentage')]
).properties(
    title='Genres Distribution'
).interactive()

alt.Chart(...)

In [84]:
credits = pd.read_csv('./The_Movies_Dataset/credits.csv', low_memory=False)

In [85]:
movies_metadata['id'] = pd.to_numeric(movies_metadata['id'], errors='coerce')
credits['id'] = pd.to_numeric(credits['id'], errors='coerce')
movies_metadata_credits = movies_metadata.merge(credits, on='id')

In [125]:
# Calculating the average rating for each movie is not enough. We need to calculate the weighted rating for each movie.
# https://math.stackexchange.com/questions/169032/understanding-the-imdb-weighted-rating-function-for-usage-on-my-own-website
# Weighted Rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C

# v = number of votes for the movie
# m = minimum votes required to be listed in the chart
# R = average rating of the movie
# C = mean vote across the whole report

C = movies_metadata_credits['vote_average'].mean()
m = movies_metadata_credits['vote_count'].quantile(0.90)

# filter out movies that have less than m votes
movies_metadata_credits = movies_metadata_credits[movies_metadata_credits['vote_count'] >= m]

# calculate the score
movies_metadata_credits['score'] = (
    (movies_metadata_credits['vote_count'] / (movies_metadata_credits['vote_count'] + m)) * movies_metadata_credits['vote_average'] +
    (m / (movies_metadata_credits['vote_count'] + m)) * C
)
movies_metadata_credits = movies_metadata_credits.sort_values('score', ascending=False)

In [126]:
# Grouping by genres
genres = movies_metadata_genres.explode('genres')
genres = genres.groupby('genres').size().reset_index(name='count')
genres = genres.sort_values('count', ascending=False)
# calculate the percentage
genres['percentage'] = genres['count'] / genres['count'].sum() * 100
genres['top_movies'] = genres['genres'].apply(lambda x: ' '.join(f"{i+1}. {title}" for i, title in enumerate(movies_metadata_genres[movies_metadata_genres['genres'].apply(lambda y: x in y)]['title'].astype(str).head(10))))

In [128]:
# plot the genres distribution and show top 10 movies as a tooltip
alt.Chart(genres).mark_arc().encode(
    theta=alt.Theta(field='percentage', type='quantitative', stack=True),
    color=alt.Color(field='genres', type='nominal'),
    tooltip=[alt.Tooltip('genres', title='Genre'), alt.Tooltip('percentage', format='.2f', title='Percentage'), alt.Tooltip('top_movies', title='Top Movies')],
    order=alt.Order('count', sort='descending')
).properties(
    title='Genres Distribution and Top Movies'
).interactive()


alt.Chart(...)